In [1]:
import numpy as np 

import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from scipy.optimize import root
import math
import copy
from tqdm.notebook import tqdm

from scipy.interpolate import splprep, splev
from scipy.spatial.distance import directed_hausdorff

In [2]:
%matplotlib notebook
%run "C:\SBU-3\Jupyter-Research\RSCR\Functions.ipynb"
%run  "C:\SBU-3\Jupyter-Research\RSCR\Plot_Compare_Prediction_Functions.ipynb"

In [3]:
data = np.load('C:\SBU-3\Jupyter-Research\RSCR\Saved Data\RSCR Prediction Journal\\cvae_beta10_pre_test.npy')

In [4]:
data.shape

(30000, 316)

In [5]:
path = data[:,0:300]
mec = data[:,300:]

In [6]:
path.shape

(30000, 300)

In [7]:
mec.shape

(30000, 16)

path = path.reshape((100,3,100))
mec = mec.reshape((100,16))

In [ ]:
path = path.reshape((1000,30,3, 100))
mec = mec.reshape((1000,30,16))

In [ ]:
path.shape

In [ ]:
len(path)

In [ ]:
HAU = []
Pre_path = []
Tru_path = []
for j in tqdm(range(len(path))):
    hau_dis = []
    pre_path = []
    tru_path = []
    path_t = path[j]
    mec_t = mec[j]
    for i in range(len(path_t)):
        try:
            predict_mec = Complete_Mec_16(mec_t[i])
            joint3_predict, curve_predict = Open_Curve_RSCR(predict_mec)
            if len(joint3_predict)<358:
                continue

            _, predict_full = Frame_Predict_Mec(predict_mec)
            frame0 = np.array([[1,0,0],[0,1,0],[0,0,1]])

            coupler_relative_pos_predict = Coupler_in_Rigidbody(predict_full, frame0)

            len2 = len(curve_predict)
            coupler_path_predict = np.zeros((len2,3))
            for j in range(len2):
                coupler_path_predict[j] = Coupler_Path(predict_full, frame0, curve_predict[j], joint3_predict[j], coupler_relative_pos_predict)

            predict_bspline = Path_Interpolate(coupler_path_predict, num_pts=100)
            predict_normalized, _, _, _ = normalizePath(predict_bspline)

            hausdorff = directed_hausdorff(path_t[i].T, predict_normalized.T)[0]
            hau_dis.append(hausdorff)
            pre_path.append(predict_normalized)
            tru_path.append(path_t[i])
        except:
            pass
    
    if not hau_dis:
        continue
    hau = np.min(hau_dis)
    inx_hau = np.argmin(hau_dis)
    pre = pre_path[inx_hau]
    tru = tru_path[inx_hau]
    
    HAU.append(hau)
    Pre_path.append(pre)
    Tru_path.append(tru)

In [ ]:
HAU = np.array(HAU)
Pre_path = np.array(Pre_path)
Tru_path = np.array(Tru_path)

good_curve = 0
for dis in HAU:
    if dis < 0.027:
        good_curve += 1
        
print('valid curve:', len(HAU))
print('good curve:', good_curve)
print('mean:',np.mean(HAU))
print('median:', np.median(HAU))

In [ ]:
np.var(HAU)

In [ ]:
np.save('cvae_hau.npy', HAU)

In [ ]:
print(np.min(hau_dis), np.argmin(hau_dis))

In [ ]:
print(np.max(hau_dis), np.argmax(hau_dis))

In [ ]:
fig= plt.figure(figsize=(10, 10))
rand = random.randint(0,len(HAU)-1)
print(HAU[rand])
ax = plt.subplot(111, projection='3d')
plotPath(Tru_path[1],color = 'red', linestyle = 'point2') #true
plotPath(Pre_path[1],color='green',linestyle='point2')
ax.set_xlim(-0.2,0.2)
ax.set_ylim(-0.2,0.2)
ax.set_zlim(-0.2,0.2)
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

In [ ]:
Tru_path = np.load('C:\SBU-3\Jupyter-Research\RSCR\Saved Data\RSCR Prediction Journal\\beta10_tru.npy')
Pre_path = np.load('C:\SBU-3\Jupyter-Research\RSCR\Saved Data\RSCR Prediction Journal\\beta10_pre.npy')

In [ ]:
fig= plt.figure(figsize=(10, 10))
rand = np.random.randint(len(Tru_path),size=16)
print(rand)
for i in range(16):
    ax = plt.subplot(4,4,i+1, projection='3d')
    plotPath(Tru_path[rand[i]],color = 'red', linestyle = 'point2') #true
    plotPath(Pre_path[rand[i]],color='green',linestyle='point2')
    ax.set_xlim(-0.2,0.2)
    ax.set_ylim(-0.2,0.2)
    ax.set_zlim(-0.2,0.2)
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')

In [ ]:
rand